In this lab we will look at Modeling an ising system with Monte Carlo.

# Ising Model 

The Ising model is a lattice of $k$ points that has a value $\sigma_k$ which can have a value of +1 or -1. The spins are allowed to interact with their neighboring spins. For two neighboring spins, the Hamiltonian between them

$$
H(\sigma)=-\sum_{\langle i j\rangle} J_{i j} \sigma_{i} \sigma_{j}
$$

where $\langle i j\rangle$ indicate a sum over only nearest neighbors.
The interaction between spins is captured in $J$.
If $J$, is positive the system will behave ferromagnetically while a negative $J$ favors antiferromagnetic interactions. 

We will start by introducing some code for this model. 

In [10]:
## think about umbrella sampling between points
## 1-D
## 2-D phase transitions. Given equations, but not derived in class. Expectd 1/2, but actual answer is 1/8th

To begin, we provide a python class for you to add your functions to. This will help keep variables organize. Initial functions have been provided to initialize your model, print the data, and plot the model if necessary.

In [8]:
class IsingModel:
    """
    Store attributes of an Ising lattice model
    Provide abstractions to conveniently manipulate lattice for simulations
    """
    def __init__(self, M, N, J, h):
        """
        Initialization.

        parameters:
            lattice is M by N sites
            J: interaction between neighbors (units: kT)
            h: background field (units: kT)
        """
        # store parameters for convenience:
        #   energetic parameters
        self.J = J
        self.h = h

        #   size of lattice
        self.M = M
        self.N = N

        # store lattice state with M by N array of -1 or 1
        # initialize each site as -1 or 1 with equal probability
        lattice_state = np.random.randint(-1, high=1, size=(M, N))
        lattice_state[lattice_state == 0] = 1
        self.lattice_state = lattice_state
    
    def print_params(self):
        """
        Print lattice attributes
        """
        print("\t%d by %d lattice".format((self.M, self.N)))
        print("\tJ = %f   (+ve means preferable )".format(self.J))
        print("\th = %f".format(self.h))

    def plot_lattice(self):
        """
        Plot lattice configuration
        """
        plt.figure()

        imgplot = plt.imshow(self.lattice_state)
        imgplot.set_interpolation('none')

        plt.xticks(range(self.N))
        plt.yticks(range(self.M))

        plt.show()

In [5]:
 class IsingModel(IsingModel):
     def flip_spin(self, i, j):
        """
        Flip spin (i, j)
        i.e. -1 ---> 1
              1 ---> -1
        """
        self.lattice_state[i, j] = - self.lattice_state[i, j]

In [1]:
## Add simple test here to make sure they are interacting with class correctly. 

In [ ]:
  class IsingModel(IsingModel):
    def calculate_energy_of_sites(self, i, j):
        """
        Calculate energy of spin (i, j)
        
        Periodic boundary conditions implemented
        """
        spin_here = self.lattice_state[i, j]  # value of spin here
        
        # value of spin above, below, left, and right of spin (i, j)
        # for each, if on boundary, we wrap around to the other side
        # of the lattice for periodic boundary conditions
        if j == 0:
            spin_above = self.lattice_state[i, self.N - 1]
        else:
            spin_above = self.lattice_state[i, j - 1]
        
        if j == self.N - 1:
            spin_below = self.lattice_state[i, 0]
        else:
            spin_below = self.lattice_state[i, j + 1]
            
        if i == self.M - 1:
            spin_right = self.lattice_state[0, j]
        else:
            spin_right = self.lattice_state[i + 1, j]
        
        if i == 0:
            spin_left = self.lattice_state[self.M - 1, j]
        else:
            spin_left = self.lattice_state[i - 1, j]
        
        return - self.h * spin_here - self.J * spin_here *\
            (spin_above + spin_below + spin_left + spin_right)

In [2]:
## add test here for correct behavior

In [9]:
  class IsingModel(IsingModel):
    def calculate_lattice_energy_per_spin(self):
        """
        Calculate energy of lattice normalized by the number of spins
        """
        E = 0.0
        for i in range(self.M):
            for j in range(self.N):
                E += self.calculate_energy_of_spin(i, j)
        # factor of two for overcounting neighboring interactions.
        # but then need to add back -1/2 h \sum s_i 
        return E / 2.0 / (self.M * self.N) -\
            self.h * np.sum(self.lattice_state) / 2.0 / (self.M * self.N)

# Monte Carlo Simulations

Theoretically, we have made a very simple model for describing interacting systems. 
Additionally, we can now start to use this model for predicting observables. 
For example, for the expectation value of the energy of an $MxN$ lattice, the expectation value would be

$$
\langle E\rangle=\sum_{\alpha} E(\alpha) P(\alpha)
$$

where $E(\alpha)$ is the energy of a fixed state $\alpha$, and  $P(\alpha)$ is the probability of being in that fixed state.
However, the number of fixed states grows as $2^N$ where $N$ is your number of lattice points.
**TODO: FIX THIS FOR N NOT BEING THE SAME ON BOTH SIDES**
This quickly becomes impractical as the lattice size grows.

#References:

code: 
    
theory: https://arxiv.org/pdf/0803.0217.pdf